In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split , GridSearchCV
from sklearn.metrics import mean_squared_error , r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale     # standardizasoyon işlemleri
from sklearn.preprocessing import StandardScaler    # standardizasoyon işlemleri
from sklearn import model_selection


from catboost import CatBoostRegressor


from warnings import filterwarnings
filterwarnings("ignore")     # önceki bölümlerde çalıştırdığımızda çıkan gereksiz uyarı mesajları çıkmasın diye

In [4]:
!pip install catboost

  Using cached catboost-1.2.5-cp312-cp312-win_amd64.whl.metadata (1.2 kB)
Using cached catboost-1.2.5-cp312-cp312-win_amd64.whl (101.1 MB)


ERROR: Could not install packages due to an OSError: [WinError 5] Erişim engellendi: 'C:\\Python312\\etc'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
df = pd.read_csv("C:\Makine Öğrenmesi-2\Hitters.csv")
df = df.dropna()

dms = pd.get_dummies(df[["League" , "Division" , "NewLeague"]])

y = df["Salary"]

x__ = df.drop(["Salary" , "League" , "Division" , "NewLeague"] , axis=1).astype("float64")
x = pd.concat([x__ , dms[["League_N" , "Division_W" , "NewLeague_N"]]] , axis=1)

x_train , x_test , y_train , y_test = train_test_split(x , y , test_size=0.25 , random_state=42)

# MODEL

In [8]:
catb_model = CatBoostRegressor().fit(x_train , y_train)

Learning rate set to 0.031674
0:	learn: 437.6430699	total: 137ms	remaining: 2m 16s
1:	learn: 431.3923642	total: 138ms	remaining: 1m 8s
2:	learn: 424.8820360	total: 140ms	remaining: 46.5s
3:	learn: 418.2514904	total: 141ms	remaining: 35.1s
4:	learn: 412.6394021	total: 143ms	remaining: 28.4s
5:	learn: 406.6247020	total: 144ms	remaining: 23.8s
6:	learn: 400.5321206	total: 146ms	remaining: 20.7s
7:	learn: 394.6683437	total: 147ms	remaining: 18.2s
8:	learn: 388.2496484	total: 148ms	remaining: 16.3s
9:	learn: 382.9448842	total: 150ms	remaining: 14.8s
10:	learn: 377.2600080	total: 151ms	remaining: 13.6s
11:	learn: 372.4829606	total: 152ms	remaining: 12.5s
12:	learn: 366.6823437	total: 153ms	remaining: 11.6s
13:	learn: 362.6076230	total: 155ms	remaining: 10.9s
14:	learn: 358.0107745	total: 157ms	remaining: 10.3s
15:	learn: 353.2802665	total: 159ms	remaining: 9.76s
16:	learn: 348.5646265	total: 160ms	remaining: 9.24s
17:	learn: 343.6407912	total: 161ms	remaining: 8.79s
18:	learn: 339.2363847	to

# TAHMİN

In [9]:
y_pred = catb_model.predict(x_test)
np.sqrt(mean_squared_error(y_pred , y_test))

351.194631344607

# MODEL TUNING (DOĞRULAMA)

***Önemli Parametreler:***
- iterations = n_estimators


In [10]:
catb_params = {"iterations": [200,500,1000] , "learning_rate": [0.001,0.1] , "depth": [3,6,8]}

In [11]:
catb_cv_model = GridSearchCV(catb_model , catb_params , cv=10 , n_jobs=-1 , verbose=2).fit(x_train , y_train)

Fitting 10 folds for each of 18 candidates, totalling 180 fits
0:	learn: 425.7900818	total: 681us	remaining: 136ms
1:	learn: 404.8723520	total: 1.21ms	remaining: 120ms
2:	learn: 387.4057666	total: 1.55ms	remaining: 102ms
3:	learn: 372.2801584	total: 1.89ms	remaining: 92.8ms
4:	learn: 358.9204229	total: 2.22ms	remaining: 86.7ms
5:	learn: 347.0083933	total: 2.57ms	remaining: 83ms
6:	learn: 336.0130818	total: 2.91ms	remaining: 80.2ms
7:	learn: 324.3923300	total: 3.23ms	remaining: 77.5ms
8:	learn: 314.8690957	total: 3.55ms	remaining: 75.3ms
9:	learn: 308.5075563	total: 3.87ms	remaining: 73.6ms
10:	learn: 298.8587285	total: 4.19ms	remaining: 72.1ms
11:	learn: 294.7655438	total: 4.53ms	remaining: 70.9ms
12:	learn: 288.0697862	total: 4.86ms	remaining: 69.9ms
13:	learn: 282.6697154	total: 5.22ms	remaining: 69.3ms
14:	learn: 277.6121667	total: 5.57ms	remaining: 68.8ms
15:	learn: 273.4383979	total: 5.92ms	remaining: 68.1ms
16:	learn: 269.1556201	total: 6.27ms	remaining: 67.5ms
17:	learn: 264.809

In [12]:
catb_cv_model.best_params_

{'depth': 3, 'iterations': 200, 'learning_rate': 0.1}

In [13]:
catb_tuned = CatBoostRegressor(depth=3 , iterations=200 , learning_rate=0.1).fit(x_train , y_train)

0:	learn: 425.7900818	total: 630us	remaining: 125ms
1:	learn: 404.8723520	total: 1.61ms	remaining: 160ms
2:	learn: 387.4057666	total: 2.06ms	remaining: 136ms
3:	learn: 372.2801584	total: 2.62ms	remaining: 128ms
4:	learn: 358.9204229	total: 3.07ms	remaining: 120ms
5:	learn: 347.0083933	total: 4.41ms	remaining: 143ms
6:	learn: 336.0130818	total: 4.99ms	remaining: 138ms
7:	learn: 324.3923300	total: 5.4ms	remaining: 130ms
8:	learn: 314.8690957	total: 5.77ms	remaining: 123ms
9:	learn: 308.5075563	total: 6.19ms	remaining: 118ms
10:	learn: 298.8587285	total: 6.58ms	remaining: 113ms
11:	learn: 294.7655438	total: 6.99ms	remaining: 110ms
12:	learn: 288.0697862	total: 7.45ms	remaining: 107ms
13:	learn: 282.6697154	total: 7.89ms	remaining: 105ms
14:	learn: 277.6121667	total: 8.33ms	remaining: 103ms
15:	learn: 273.4383979	total: 8.78ms	remaining: 101ms
16:	learn: 269.1556201	total: 9.19ms	remaining: 98.9ms
17:	learn: 264.8098704	total: 9.61ms	remaining: 97.2ms
18:	learn: 261.6700768	total: 10.1ms	r

In [14]:
y_pred = catb_tuned.predict(x_test)
np.sqrt(mean_squared_error(y_pred, y_test))

344.3125832615482